# 랜덤포레스트 적용

## corr() 결과 양의 값으로 나온 변수중 가장 의미 있는 3가지(개인상점, 플레이시간, 낚시)와 생존

In [1]:
# https://github.com/Swalloow/Kaggle/blob/master/Titanic%20Survivors/Titanic%20RandomForest.ipynb/
# 설명 : https://swalloow.github.io/decison-randomforest
import numpy as np
import pandas as pd


## Load Data

In [2]:

activity_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_label.csv')
label_raw = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_activity.csv')


In [3]:
activity_raw.dtypes

acc_id             int64
survival_time      int64
amount_spent     float64
dtype: object

In [4]:
activity = activity_raw
label = label_raw

In [5]:
act_label = label.merge(activity, on ='acc_id', how='inner')

In [6]:
act_label_grp = act_label.groupby('acc_id',as_index=False).sum()

In [7]:
print(act_label_grp.shape)
act_label_grp.head()

(40000, 18)


,acc_id,day,char_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,survival_time,amount_spent
0,2,406,6540744,91.533663,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,-0.008746,0.0,1792,0.000000
1,5,409,3851049,6.116132,3.236452,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.145146,0.0,1080,0.000000
2,8,1764,26228439,18.797802,31.172609,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.0,4.807823,0.000000,0.020353,0.0,8512,2.701235
3,17,760,8548136,28.464708,30.456852,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,-0.027696,0.0,3328,3.673394
4,20,610,5571452,25.588043,34.100184,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,-0.370192,0.0,2432,1.981197


In [8]:
data = act_label_grp.drop(['day','char_id','amount_spent'], axis =1)
data['survived'] = np.where(data['survival_time'] == 64, 'survived','death')
data.head()

,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,survival_time,survived
0,2,91.533663,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,84.900753,-0.008746,0.0,1792,death
1,5,6.116132,3.236452,2.155434,0.695285,0.111055,9,0.245883,0.247337,0.0,3.442909,0.000000,0.145146,0.0,1080,death
2,8,18.797802,31.172609,31.489136,0.000000,1.561055,2,91.960416,92.504000,0.0,4.807823,0.000000,0.020353,0.0,8512,death
3,17,28.464708,30.456852,9.255848,8.823094,2.322365,21,1.721184,1.731358,0.0,13.990954,0.000000,-0.027696,0.0,3328,death
4,20,25.588043,34.100184,0.274902,0.080060,0.083483,8,0.983534,0.989348,0.0,4.295766,2.917510,-0.370192,0.0,2432,death


In [9]:
#훈련세트, 테스트세트 나누기
from sklearn.model_selection import train_test_split

x = data.iloc[:,:-1]
x.drop('survival_time', axis=1, inplace = True)
x.drop('acc_id', axis=1, inplace = True)
y = data.loc[:,'survived']

train_x, test_x, train_y, test_y = train_test_split(x,y, test_size = 0.3)

train_x.shape, test_x.shape, train_y.shape, test_y.shape


((28000, 13), (12000, 13), (28000,), (12000,))

## DecisionTree

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
decision = DecisionTreeClassifier(max_depth = 3, random_state = 7).fit(train_x,train_y)
print(decision.score(test_x, test_y))

0.9989166666666667


## RandomForest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
import graphviz

In [20]:
forst = RandomForestClassifier(n_estimators=500)
forst.fit(train_x,train_y)
print(forst.score(test_x, test_y))

0.9990833333333333


In [24]:
export_graphviz(
    forst,
    feature_names = x.columns,
    class_names = ['Death', 'Survived'],
    out_file = "C:/Users/CPB06GameN/Desktop/PROGRAM/big/game.dot",
    impurity=True, 
    filled=True)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:

with open("C:/Users/CPB06GameN/Desktop/PROGRAM/big/game.dot") as f:
#     dot_graph = str(open("game.dot", "rb").read(), 'utf8')
#     dot_graph = f.read()
    dot_graph = str(open("C:/Users/CPB06GameN/Desktop/PROGRAM/big/game.dot", "rb").read(), 'utf8')

graphviz.Source(dot_graph)  # gini(지니)값은 불순도를 의미 -> 값이 작을수록 순도가 높음.



In [81]:
graph = Source(forst.export_graphviz(estimator, out_file=None
   , feature_names=x.columns, class_names=['survived','death'] 
   , filled = True))
display(SVG(graph.pipe(format='svg')))

AttributeError: 'RandomForestClassifier' object has no attribute 'export_graphviz'